In [1]:
# ==========================================
# File: 03_final_training_svm.ipynb (Train+Val 合并训练版)
# ==========================================

import os
import json
import joblib
import pandas as pd
import numpy as np
from sklearn.svm import SVR
from sklearn.preprocessing import PowerTransformer, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import r2_score

if 'notebooks' in os.getcwd(): os.chdir('..')

# 1. 读取数据
print(">>> [Step 1] 读取数据...")
try:
    df_train = pd.read_csv('data/processed/train_data.csv')
    df_val = pd.read_csv('data/processed/val_data.csv')
    df_test = pd.read_csv('data/processed/test_data.csv')
except FileNotFoundError:
    raise FileNotFoundError("❌ 找不到数据！")

# 准备数据函数
def split_xy(df):
    return df['Target_Log1o2'], df.drop(columns=['Target_Log1o2', 'SMILES'])

y_train, X_train = split_xy(df_train)
y_val, X_val = split_xy(df_val)
y_test, X_test = split_xy(df_test)

# ========================================================
# ★★★ 核心修改：合并 Train 和 Val ★★★
# ========================================================
print(">>> [Step 2] 合并 Train + Val 进行全量训练...")
X_combined = pd.concat([X_train, X_val], axis=0).reset_index(drop=True)
y_combined = pd.concat([y_train, y_val], axis=0).reset_index(drop=True)

print(f"   - 原 Train 样本数: {len(X_train)}")
print(f"   - 原 Val   样本数: {len(X_val)}")
print(f"   - 合并后训练集数: {len(X_combined)} (数据量增加，模型更稳！)")

# 2. 读取参数 (来自 02 的调参结果)
try:
    with open('results/models/best_params_svm.json', 'r') as f:
        params = json.load(f)
    print("✅ 已加载最佳参数")
    best_svr = SVR(
        C=params['regressor__svr__C'],
        gamma=params['regressor__svr__gamma'],
        epsilon=params['regressor__svr__epsilon']
    )
except:
    print("⚠️ 警告：未找到参数文件，使用默认参数！")
    best_svr = SVR(C=50, gamma=0.01)

# 3. 构建模型 (无 RFECV)
inner_pipe = Pipeline([
    ('yeo', PowerTransformer()),
    ('scaler', RobustScaler()),
    ('svr', best_svr)
])
model = TransformedTargetRegressor(regressor=inner_pipe, transformer=RobustScaler())

# ★★★ 训练：使用合并后的数据 ★★★
model.fit(X_combined, y_combined)

# 4. 预测与评估
print(">>> [Step 3] 生成预测结果...")
# 注意：现在 Train 和 Val 都是“训练内数据”了
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)
y_pred_test = model.predict(X_test) # 这是唯一的“外样本”预测

print("\n" + "="*30)
print("【模型表现】(注：Val 分数变高是正常的，因为它现在参与了训练)")
print(f"🔵 Train R2: {r2_score(y_train, y_pred_train):.4f}")
print(f"🟢 Val   R2: {r2_score(y_val, y_pred_val):.4f}")
print("-" * 20)
print(f"🔴 Test  R2: {r2_score(y_test, y_pred_test):.4f} <--- 最终看这个！")
print("="*30 + "\n")

# 5. 保存结果
os.makedirs('results/predictions', exist_ok=True)
os.makedirs('results/models', exist_ok=True)

df_train['Predicted'] = y_pred_train; df_train.to_csv('results/predictions/train_predictions.csv', index=False)
df_val['Predicted'] = y_pred_val;     df_val.to_csv('results/predictions/val_predictions.csv', index=False)
df_test['Predicted'] = y_pred_test;   df_test.to_csv('results/predictions/final_predictions.csv', index=False)
joblib.dump(model, 'results/models/best_svm_model.pkl')

print("💾 结果已保存！去运行 04 看图吧！")

>>> [Step 1] 读取数据...
>>> [Step 2] 合并 Train + Val 进行全量训练...
   - 原 Train 样本数: 105
   - 原 Val   样本数: 23
   - 合并后训练集数: 128 (数据量增加，模型更稳！)
✅ 已加载最佳参数
>>> [Step 3] 生成预测结果...

【模型表现】(注：Val 分数变高是正常的，因为它现在参与了训练)
🔵 Train R2: 0.7764
🟢 Val   R2: 0.8574
--------------------
🔴 Test  R2: 0.6348 <--- 最终看这个！

💾 结果已保存！去运行 04 看图吧！
